In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import logging
import glob
import tqdm
import torch.utils.data as data
from torchvision import transforms
import matplotlib.pyplot as plt
from torch.nn import Conv2d, MaxPool2d, Flatten, Linear, Sequential
from sklearn import preprocessing
import category_encoders as ce
import warnings
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings('ignore')
import pickle
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
import scipy.sparse as sparse
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict, Counter
from torch.utils.data import Dataset, DataLoader
import os, pickle
from collections import defaultdict, Counter
import tqdm
# estimate runningtime : EGES----16 hours; FM----6 hours; whole program: 24 hours

ModuleNotFoundError: No module named 'torch'

In [2]:
# read the files
customer_df = pd.read_csv('./data/customers.csv') 
sub_df = pd.read_csv('./data/sample_submission.csv') 
train_df = pd.read_csv('./data/transactions_train.csv', dtype={"article_id": str}) 
article_df = pd.read_csv('./data/articles.csv', dtype={"article_id": str}) 
train_df['t_dat'] = pd.to_datetime(train_df['t_dat'])

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
article_df.head(5)

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,Unnamed: 25,Unnamed: 26
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.,NaN,NaN
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.,NaN,NaN
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.,NaN,NaN
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde...",NaN,NaN
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde...",NaN,NaN


In [3]:
# convert customer_id into index to reduce the memory usage
Customer_index = customer_df['customer_id'].unique().tolist()

customer_to_index_map = {customer: customer_index for customer_index, customer in enumerate(Customer_index)}
index_to_customer_map = {customer_index: customer for customer_index, customer in enumerate(Customer_index)}
pickle.dump(customer_to_index_map, open(f'customer_to_index_map.pkl','wb'))

In [4]:
# if the road already has customer_to_index_map.pkl, use this to repeat the result.
customer_to_index_map = pickle.load(open("customer_to_index_map.pkl", "rb"))

In [5]:
# convert customer_id to user_id
train_df['user_id'] = train_df['customer_id'].map(customer_to_index_map)
train_df.drop(['customer_id'], axis = 1, inplace = True)

###### preprocessing article_df

In [6]:
def preprocess_article_df(article_df):
    # when readjing article information, it misses 0 in the id columns
    article_id_df = article_df.apply(lambda x: '0' + str(x[0]), axis = 1)
    article_df['article_id'] = article_id_df
    article_df.drop(['Unnamed: 25', 'Unnamed: 26'], axis = 1, inplace = True)
    # drop some useless columns
    article_df = article_df.drop(['graphical_appearance_no','colour_group_code', \
                          'perceived_colour_value_id', 'perceived_colour_master_id',\
                          'department_no', 'index_code', 'index_group_no', 'section_no', \
                          'garment_group_no'], axis = 1)
    # combine "department features"
    article_df['department'] = article_df['department_name'].astype(str) \
                            + '_' + article_df['garment_group_name'].astype(str)
    article_df.drop(['department_name', 'garment_group_name'], axis = 1, inplace = True)
    # combine "index features"
    article_df['index_features'] = article_df['index_name'].astype(str) + \
                    '_' + article_df['index_group_name'].astype(str)
    article_df.drop(['index_name', 'index_group_name'], axis = 1, inplace = True)
    # combine "color feature"
    article_df['color'] = article_df['colour_group_name'].astype(str) \
                        + '_' + article_df['perceived_colour_value_name'].astype(str) + '_' + \
                        article_df['perceived_colour_master_name'].astype(str)
    article_df.drop(['colour_group_name', 'perceived_colour_value_name', \
                  'perceived_colour_master_name'], axis = 1, inplace = True)
    # combine "product features"
    article_df['product_features'] = article_df['product_type_name'].astype(str) + '_' +  \
                             article_df['product_group_name'].astype(str)
    article_df.drop(['product_code', 'product_type_no', 'prod_name', \
                             'product_type_name', 'product_group_name'], axis = 1, inplace = True)
    # convert all the object variables into category type
    categorical_columns = article_df.select_dtypes(include = 'object').columns
    for categorical_column in categorical_columns:
        article_df[categorical_column] = pd.Categorical(article_df[categorical_column])
    # drop detail_desc since it has some null 
    article_df = article_df.drop('detail_desc', axis = 1)
    article_df = article_df.reset_index(drop = True)
    return article_df
article_df = preprocess_article_df(article_df)

In [7]:
# save the processed dataframe
pickle.dump(article_df, open(f'article_df.pkl','wb'))
pickle.dump(train_df, open(f'train_df.pkl','wb'))

In [ ]:
# read the article_df and train_df
article_df = pickle.load(open("article_df.pkl", "rb"))
train_df = pickle.load(open("train_df.pkl", "rb"))
'''
this is fast when after using the model

'''

In [9]:
# we use data after 2020-3-22 to apply EGES
transactions = train_df[train_df['t_dat'] >= '2020-03-22']
transactions = transactions.merge(article_df, on = 'article_id', how = 'inner')

In [10]:
# drop 'price', 'sales_channel_id' since they are related to the dynamically changed and could not be used in prediction
transactions = transactions.drop(['price', 'sales_channel_id'], axis = 1)
transactions.columns

Index(['t_dat', 'article_id', 'user_id', 'graphical_appearance_name',
       'section_name', 'department', 'index_features', 'color',
       'product_features'],
      dtype='object')

In [11]:
# label_list are the list that used in sparse
label_list = ['graphical_appearance_name',
              'section_name',
              'product_features',
              'department',
              'index_features',
              'color',
              'article_id'
              ]


In [12]:
# label encoder
lbe_dict = {}

# label encoder + 1, in case of padding
for target_category_col in label_list:
    label_encoder = LabelEncoder()
    transactions[target_category_col] = pd.Categorical(transactions[target_category_col])
    transactions[target_category_col] = label_encoder.fit_transform(transactions[target_category_col]) + 1
    lbe_dict[target_category_col] = label_encoder
pickle.dump(lbe_dict, open(f'label_dict.pkl','wb'))
pickle.dump(transactions, open(f'transactions.pkl','wb'))

In [ ]:
lbe_dict = pickle.load(open("label_dict.pkl", "rb"))
transactions = pickle.load(open("transactions.pkl", "rb"))
'''
this is fast when after using the model

'''

In [14]:
# vocab_dict is the number of nuniques of each features + 1
vocab_dict = {feat:len(lbe_dict[feat].classes_) + 1 for feat in lbe_dict}
vocab_dict

{'graphical_appearance_name': 31,
 'section_name': 57,
 'product_features': 128,
 'department': 244,
 'index_features': 11,
 'color': 282,
 'article_id': 51479}

In [15]:
# product is the articles bought in this time periods
product = transactions[label_list].drop_duplicates(subset = label_list)
product.shape

(51478, 7)

In [16]:
pickle.dump(product, open(f'product.pkl','wb'))
pickle.dump(vocab_dict, open(f'vocab_dict.pkl','wb'))

In [4]:
# read the vocab_dict and product

product = pickle.load(open("product.pkl", "rb"))
vocab_dict = pickle.load(open("vocab_dict.pkl", "rb"))
'''
this is fast when after using the model

'''

'\nthis is fast when after using the model\n\n'

#### preprocessing on transactions

In [18]:
transactions['user_id'] = pd.Categorical(transactions['user_id'])
transactions['t_dat'] = pd.to_datetime(transactions['t_dat'])

In [19]:
# only keep the goods id in the whole transactions
product = product[product['article_id'].isin(set(transactions['article_id']))]

In [20]:
good_counts = Counter(transactions['article_id'].tolist()).items()
good_counts = sorted(good_counts, key = lambda x: x[0])
counts = np.array([wc[1] for wc in good_counts])


In [21]:
pickle.dump(good_counts, open(f'good_counts.pkl', 'wb')) 

In [22]:
good_counts = pickle.load(open("good_counts.pkl", "rb"))
'''
this is fast when after using the model

'''

'\nthis is fast when after using the model\n\n'

In [23]:
df_corpus = transactions.groupby('user_id')['article_id'].agg(list)
df_corpus.head(2)

user_id
0                                               [4104]
1    [580, 16128, 34272, 5591, 5591, 5599, 3592, 30...
Name: article_id, dtype: object

In [24]:
def generate_context_pairs(corpus, window = 5):
    """
    copurs: df_pairs
    window: the max distance from center word to side words, in this case articles
    """
    all_pairs = []
    for k in tqdm.tqdm(range(len(corpus))):
        sent = corpus[k]
        if len(sent) < 2: continue
        for i in range(len(sent)):
            for j in range(max(i - window, 0), min(i + window + 1, len(sent))):
                if i!=j:
                    all_pairs.append([sent[i], sent[j]])
    return all_pairs
all_pairs = generate_context_pairs(df_corpus.values, 5)

100%|███████████████████████████████████████████████████████████████████████| 748053/748053 [00:32<00:00, 23247.93it/s]


In [25]:
df_pair = pd.DataFrame(all_pairs, columns=['article_id', 'label'])
print(df_pair.shape)
df_pair.head(2)

(62463592, 2)


,article_id,label
0,580,16128
1,580,34272


In [26]:
df_pair = df_pair.merge(product, on = 'article_id', how = 'left')
df_pair.head()

,article_id,label,graphical_appearance_name,section_name,product_features,department,index_features,color
0,580,16128,26,52,99,191,8,8
1,580,34272,26,52,99,191,8,8
2,580,5591,26,52,99,191,8,8
3,580,5591,26,52,99,191,8,8
4,580,5599,26,52,99,191,8,8


In [28]:
df_pair.to_pickle('df_pairs.pkl')

In [29]:
df_pair = pickle.load(open("df_pairs.pkl", "rb"))
'''
this is fast when after using the model

'''

'\nthis is fast when after using the model\n\n'

# EGES

In [30]:
DATA_PATH = 'data_cache/'
def create_embedding_matrix(sparse_columns, varlen_sparse_columns, embed_dim,
                            init_std = 0.0001, padding = True, device = 'cpu', mode = 'mean'):
    # sparse_columns => dict{'name':vocab_size}
    # Return nn.ModuleDict: for sparse features, {embedding_name: nn.Embedding}
    padding_idx = 0 if padding else None
    sparse_embedding_dict = {
        feat: nn.Embedding(sparse_columns[feat], embed_dim, padding_idx=padding_idx)
                             for feat in sparse_columns
    }
    
    if varlen_sparse_columns:
        varlen_sparse_embedding_dict = {
            feat:nn.EmbeddingBag(varlen_sparse_columns[feat], embed_dim, padding_idx=padding_idx,
                                 mode=mode) for feat in varlen_sparse_columns
        }
        sparse_embedding_dict.update(varlen_sparse_embedding_dict)
        
    embedding_dict = nn.ModuleDict(sparse_embedding_dict)
    
    for tensor in embedding_dict.values():
        nn.init.normal_(tensor.weight, mean=0, std=init_std)
        # nn.init.kaiming_uniform_(tensor.weight, mode='fan_in', nonlinearity='relu')

    return embedding_dict.to(device)

In [31]:
class EGES(nn.Module):
    def __init__(self, sparse_dict, varlen_sparse_dict = None, target_col = 'article_id',
                 n_embed = 64, k_side = 6, noise_dist = None, device = 'cpu', padding=True):
        """sparse_dict: dict, {feature_name: vocab_size}
        """
        super().__init__()
        self.n_embed = n_embed
        self.k_side = k_side # number of features (except the id feature)
        self.device = device
        self.padding = padding
        self.target_col = target_col
        self.features = list(sparse_dict.keys())
        if varlen_sparse_dict:
            self.features = self.features + list(varlen_sparse_dict.keys())
        self.sample_word_offset = 1 if padding else 0
        # input embedding dict, include item and side info
        self.input_embedding_dict = create_embedding_matrix(
            sparse_dict, varlen_sparse_dict, n_embed,
            init_std = 0.0001, padding = padding, device = device, mode = 'mean')
        self.out_embed = nn.Embedding(sparse_dict[target_col], n_embed,
                                      padding_idx = 0 if padding else None)
        self.attn_embed = nn.Embedding(sparse_dict[target_col], k_side + 1, 
                                       padding_idx = 0 if padding else None)
        
        # Initialize out embedding tables with uniform distribution
        nn.init.normal_(self.out_embed.weight, mean = 0, std = 0.0001)
        nn.init.normal_(self.attn_embed.weight, mean = 0, std = 0.0001)

        if noise_dist is None:
            # sampling words uniformly
            self.noise_dist = torch.ones(self.n_vocab)
        else:
            self.noise_dist = noise_dist
        self.noise_dist = self.noise_dist.to(device)

    def forward_input(self, input_dict):
        # return input vector embeddings
        embed_lst = []
        for col in self.features:
            if col in input_dict:
                input_vector = self.input_embedding_dict[col](input_dict[col])
                embed_lst.append(input_vector)

        batch_size = input_vector.shape[0]
        # embeds => [batch_size, k_side+1, n_embed]
        embeds = torch.cat(embed_lst, dim = 1).reshape(batch_size, self.k_side + 1, self.n_embed)
        
        # attation => [batch_size, k_side+1]
        attn_w = self.attn_embed(input_dict[self.target_col])
        attn_w = torch.exp(attn_w)
        attn_s = torch.sum(attn_w, dim = 1).reshape(-1, 1)
        attn_w = (attn_w / attn_s).reshape(batch_size, 1, self.k_side + 1) # 归一化
        
        # attw => [batch_size, 1, k_side+1]
        # embeds => [batch_size, k_side+1, embed_size]
        # matmul out => [batch_size, 1, embed_size]
        input_vector = torch.matmul(attn_w, embeds).squeeze(1)
        
        return input_vector

    def forward_output(self, output_words):
        # return output vector embeddings 
        output_vector = self.out_embed(output_words)
        return output_vector
    
    def forward_noise(self, batch_size, n_samples):
        """Generate noise vectors with shape [batch_size, n_samples, n_embed]
        """
        # sample words from our noise distribution 
        noise_words = torch.multinomial(self.noise_dist, batch_size * n_samples, 
                                        replacement = True) + self.sample_word_offset
        noise_vector = self.out_embed(noise_words).view(batch_size, n_samples, self.n_embed)
        
        return noise_vector
    
    def forward_cold(self, input_dict):
        # return input vector embeddings
        embed_lst = []
        for col in self.features:
            if col in input_dict:
                input_vector = self.input_embedding_dict[col](input_dict[col])
                embed_lst.append(input_vector)

        batch_size = input_vector.shape[0]
        # embeds => [batch_size, k_side, n_embed]
        embeds = torch.cat(embed_lst, dim = 1).reshape(batch_size, self.k_side, self.n_embed)
        return torch.mean(embeds, dim = 1)


class NegativeSamplingLoss(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, input_vectors, output_vectors, noise_vectors):
        batch_size, embed_size = input_vectors.shape
        
        # input vectors should be a batch of column vectors
        input_vectors = input_vectors.view(batch_size, embed_size, 1)
        
        # output vectors should be a batch of row vectors
        output_vectors = output_vectors.view(batch_size, 1, embed_size)
        
        # bmm = batch matrix multiplication
        # target words log-sigmoid loss
        out_loss = torch.bmm(output_vectors, input_vectors).sigmoid().log()
        
        # negative sampling words log-sigmoid loss
        # negative words sigmoid optmize to small, thus here noise_vectors.neg()
        noise_loss = torch.bmm(noise_vectors.neg(), input_vectors).sigmoid().log()
        # sum the losses over the sample of noise vectors
        noise_loss = noise_loss.squeeze().sum(1)
        
        # sum target and negative loss
        return -(out_loss + noise_loss).mean()


class TextData(Dataset):
    def __init__(self, df, sparse_columns, varlen_sparse_columns = [], device = 'cpu'):
        self.sparse_columns = sparse_columns
        self.varlen_sparse_columns = varlen_sparse_columns
        self.device = device
        self.data = {
            col:df[col].values for col in sparse_columns
        }
        if varlen_sparse_columns:
            for col in varlen_sparse_columns:
                self.data[col] = np.vstack(df[col].values)

        self.data_num = len(df)
    
    def __len__(self):
        return self.data_num
    
    def __getitem__(self, idx):
        data_dic = {}
        for col in self.sparse_columns:
            data_dic[col] = torch.tensor(self.data[col][idx]).long() #.to(self.device)
        if self.varlen_sparse_columns:
            for col in self.varlen_sparse_columns:
                data_dic[col] = torch.tensor(self.data[col][idx, :]).long() #.to(self.device)

        return data_dic

In [32]:
counts = np.array([wc[1] for wc in good_counts])

In [33]:
noise_dist = torch.from_numpy(counts ** (0.75) / np.sum(counts ** (0.75)))

In [34]:
sparse_columns = ['article_id', 'label', 'graphical_appearance_name', 'section_name', 'department',\
                 'index_features', 'color', 'product_features']

In [36]:
logger = logging.getLogger(__name__)
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
device = 'gpu'
if device == 'gpu' and torch.cuda.is_available():
    # print('cuda ready...')
    device = 'cuda:0'
else:
    device = 'cpu'
textdata = TextData(df_pair, sparse_columns = sparse_columns) 
textloader = DataLoader(textdata,
                        batch_size = 10000,
                        shuffle = True,
                        num_workers = 0,
                        drop_last = False,
                        pin_memory = True)

embedding_dim_eges = 100
model = EGES(vocab_dict, n_embed = embedding_dim_eges, k_side = 6, target_col = 'article_id',
             noise_dist = noise_dist, device = device, padding = True).to(device)
criterion = NegativeSamplingLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
# optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01)

epoch = 3
for e in range(epoch):
    divide = 0
    for i, data_dic in enumerate(textloader):
        # input, output and noise vectors
        data_dic = {feat:data_dic[feat].to(device) for feat in data_dic}
        input_vectors = model.forward_input(data_dic)
        output_vectors = model.forward_output(data_dic['label'])
        noise_vectors = model.forward_noise(data_dic['label'].shape[0], 10)
        # negative sampling loss
        loss = criterion(input_vectors, output_vectors, noise_vectors)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        divide += 1
        if divide % 1000 == 0:
            logger.info(f'Divide {divide}/ success')

    logger.info(f'Epoch {e+1}/{epoch} Step {i} Loss = {loss}')

05/04/2022 21:51:22 - INFO - __main__ -   Divide 1000/ success
05/04/2022 22:02:16 - INFO - __main__ -   Divide 2000/ success
05/04/2022 22:12:53 - INFO - __main__ -   Divide 3000/ success
05/04/2022 22:23:43 - INFO - __main__ -   Divide 4000/ success
05/04/2022 22:34:24 - INFO - __main__ -   Divide 5000/ success
05/04/2022 22:44:52 - INFO - __main__ -   Divide 6000/ success
05/04/2022 22:47:25 - INFO - __main__ -   Epoch 1/3 Step 6246 Loss = 2.4549567699432373
05/04/2022 22:57:53 - INFO - __main__ -   Divide 1000/ success
05/04/2022 23:08:18 - INFO - __main__ -   Divide 2000/ success
05/04/2022 23:18:45 - INFO - __main__ -   Divide 3000/ success
05/04/2022 23:29:38 - INFO - __main__ -   Divide 4000/ success
05/04/2022 23:40:07 - INFO - __main__ -   Divide 5000/ success
05/04/2022 23:50:29 - INFO - __main__ -   Divide 6000/ success
05/04/2022 23:53:01 - INFO - __main__ -   Epoch 2/3 Step 6246 Loss = 2.3850114345550537
05/05/2022 00:03:34 - INFO - __main__ -   Divide 1000/ success
05/05

In [37]:
# save the model
torch.save(model.state_dict(), f'{DATA_PATH}/model.bin')

In [38]:
# get the embedding of each item
df_item = df_pair.drop_duplicates(subset = ['article_id'])
print(df_item.shape)
df_item.head(2)

(51405, 8)


,article_id,label,graphical_appearance_name,section_name,product_features,department,index_features,color
0,580,16128,26,52,99,191,8,8
5,16128,580,26,52,9,191,8,8


In [39]:
logger = logging.getLogger(__name__)
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
device = 'gpu'
if device == 'gpu' and torch.cuda.is_available():
    # print('cuda ready...')
    device = 'cuda:0'
else:
    device = 'cpu'
textdata1 = TextData(df_item, sparse_columns = sparse_columns)
textloader1 = DataLoader(textdata1, 
                         batch_size = 10000, 
                         shuffle = False, 
                         drop_last = False, 
                         pin_memory = True)
embedding_dim_eges = 100
model = EGES(vocab_dict, n_embed = embedding_dim_eges, k_side = 6, target_col = 'article_id',
             noise_dist = noise_dist, device = device, padding = True).to(device)
state_dic = torch.load(f'{DATA_PATH}/model.bin')
model.load_state_dict(state_dic)
model = model.eval()

epoch = 3
emb_vectors = []
with torch.no_grad():
    for i, data_dic in enumerate(textloader1):
        # input, output and noise vectors
        data_dic = {feat:data_dic[feat].to(device) for feat in data_dic}
        input_vectors = model.forward_input(data_dic)
        emb_vectors.append(input_vectors.detach().cpu().numpy())

In [40]:
vocab_emb = dict(zip(df_item['article_id'].tolist(), np.vstack(emb_vectors)))
embed_1 = pd.DataFrame(data = vocab_emb).T.reset_index()
embed_1 = embed_1.rename(columns = {'index': 'article_id'})
len(vocab_emb)

51405

In [41]:
cold_article = transactions[~ transactions['article_id'].isin(embed_1['article_id'])]
df_item2 = cold_article.drop_duplicates(subset=['article_id'])[['graphical_appearance_name', \
                                                     'section_name', 'department',\
                                                     'index_features', 'color', 'product_features']]
textdata2 = TextData(df_item2, sparse_columns = ['graphical_appearance_name', \
                                                     'section_name', 'department',\
                                                     'index_features', 'color', 'product_features']) 
textloader2 = DataLoader(textdata2,
                        batch_size = 10000,
                        shuffle = False,
                        drop_last = False,
                        pin_memory = True)

model = EGES(vocab_dict, n_embed = embedding_dim_eges, k_side = 6, target_col = 'article_id',
             noise_dist = noise_dist, device = device, padding = True).to(device)
state_dic = torch.load(f'{DATA_PATH}/model.bin')
model.load_state_dict(state_dic)
model = model.eval()

epoch = 3
cold_vectors = []
with torch.no_grad():
    for i, data_dic in enumerate(textloader2):
        # input, output and noise vectors
        data_dic = {feat:data_dic[feat].to(device) for feat in data_dic}
        input_vectors = model.forward_cold(data_dic)
        cold_vectors.append(input_vectors.detach().cpu().numpy())

In [42]:
cold_emb = dict(zip(cold_article.drop_duplicates(subset = ['article_id'])['article_id'].tolist(), np.vstack(cold_vectors)))
embed_2 = pd.DataFrame(data = cold_emb).T.reset_index()
embed_2 = embed_2.rename(columns = {'index': 'article_id'})
len(cold_emb)

73

In [43]:
pickle.dump(vocab_emb, open(f'vocab_emb.pkl','wb'))
pickle.dump(cold_emb, open(f'cold_emb.pkl','wb'))

In [44]:
vocab_emb = pickle.load(open("vocab_emb.pkl", "rb"))
cold_emb = pickle.load(open("cold_emb.pkl", "rb"))
'''
this is fast when after using the model

'''

'\nthis is fast when after using the model\n\n'

In [46]:
embed = pd.concat([embed_1, embed_2], axis = 0)
embed.shape

(51478, 101)

In [47]:
pickle.dump(embed, open(f'embed.pkl','wb'))

In [5]:
embed = pickle.load(open("embed.pkl", "rb"))
'''
this is fast when after using the model

'''

'\nthis is fast when after using the model\n\n'

In [6]:
embed

,article_id,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,580,0.084992,-0.120273,0.003963,-0.081431,-0.116888,-0.023568,-0.174894,0.103128,0.017729,...,0.026659,0.147948,0.001756,-0.230078,0.018663,-0.113092,-0.062846,0.042676,-0.033684,0.087502
1,16128,0.055172,-0.033577,0.009442,-0.084133,-0.068044,-0.036341,-0.103877,0.088267,-0.008914,...,-0.007283,0.137015,0.032505,-0.141099,0.036039,-0.196943,0.002166,0.098618,-0.005751,0.048917
2,34272,0.088780,-0.073143,-0.063883,0.054561,0.059336,-0.081845,0.059603,-0.139398,-0.105134,...,-0.035409,0.003863,0.023866,0.002993,-0.003376,0.093379,-0.027540,-0.091256,-0.060654,0.043634
3,5591,0.022267,-0.161352,0.018141,0.012872,0.112759,0.028385,-0.096220,0.130043,0.102090,...,-0.024699,0.216010,-0.019542,-0.090856,0.068689,-0.108800,0.022696,0.093130,0.031740,0.112459
4,5599,0.006237,-0.131394,-0.019968,0.039123,0.081398,-0.125605,-0.125922,0.041584,0.110282,...,0.106852,0.215627,0.048003,-0.004737,0.010335,0.040623,0.028842,0.018428,0.005410,0.127457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,28318,0.011530,-0.044742,0.002643,-0.011702,0.091530,-0.037206,-0.016242,0.066128,0.023217,...,-0.017675,-0.031758,0.059508,0.005869,0.013107,-0.189967,-0.009812,0.027702,0.037468,0.141796
69,23392,0.067459,-0.272700,-0.124268,-0.135245,-0.057184,-0.217716,-0.142877,0.260113,-0.021388,...,0.106918,-0.163500,0.193337,-0.124138,0.162893,-0.508588,0.244683,-0.177073,0.164317,-0.086847
70,50497,0.243048,-0.035740,-0.091096,0.140926,-0.088643,0.022642,-0.043034,0.011003,0.090474,...,0.043488,-0.159995,0.158959,0.127475,0.118843,-0.187187,0.058009,0.130073,-0.149105,0.024234
71,48844,0.022625,0.115115,-0.038445,0.008991,0.056205,-0.019346,0.052624,0.079166,0.119205,...,-0.005914,0.195732,0.041930,0.022495,0.053857,-0.074148,0.088767,0.260374,0.186994,0.192870


In [ ]:
'''
sub_customer_id = sub_df.customer_id.map(customer_to_index_map)
sub_customer = sub_df.customer_id
embed_matrix = np.array(embed.sort_values('article_id').set_index('article_id'))
df_corpus_pd = pd.DataFrame(df_corpus)
customer_id_list = []
pred_id_list = []
for index, row in tqdm.tqdm(df_corpus_pd.iterrows()):
    customer_id = index
    item_list_ = row[0]
    sim_ = np.zeros((1, embed.shape[0]))
    len_sim_ = len(sim_)
    for item_ in item_list_:
        sim_ += cosine_similarity(embed_matrix[item_ - 1].reshape(1, embedding_dim_eges), embed_matrix)
    sorted_sim_index_ = np.argsort(sim_)[0]
    pred_list = sorted_sim_index_[len(sorted_sim_index_) - 12 - len_sim_:len(sorted_sim_index_) - len_sim_]
    result_list = [lbe_dict['article_id'].inverse_transform([item_id])[0] for item_id in pred_list]
    preds = ' '.join(result_list)
    customer_id_list.append(sub_customer[customer_id])
    pred_id_list.append(preds)

'''

In [6]:
import os
import glob
import reco
from tqdm import tqdm
import datetime
from collections import Counter

In [7]:
transactions["t_dat"] = pd.to_datetime(transactions["t_dat"])
train1 = transactions[(transactions["t_dat"] >= datetime.datetime(2020,8,23)) & (transactions['t_dat'] < datetime.datetime(2020,9,23))]
train2 = transactions[(transactions["t_dat"] >= datetime.datetime(2020,7,23)) & (transactions['t_dat'] < datetime.datetime(2020,8,23))]
train3 = transactions[(transactions["t_dat"] >= datetime.datetime(2020,6,23)) & (transactions['t_dat'] < datetime.datetime(2020,7,23))]
train4 = transactions[(transactions["t_dat"] >= datetime.datetime(2020,5,23)) & (transactions['t_dat'] < datetime.datetime(2020,6,23))]
# trainx is the month dataset
positive_items_per_user1 = train1.groupby(['user_id'])['article_id'].apply(list)
positive_items_per_user2 = train2.groupby(['user_id'])['article_id'].apply(list)
positive_items_per_user3 = train3.groupby(['user_id'])['article_id'].apply(list)
positive_items_per_user4 = train4.groupby(['user_id'])['article_id'].apply(list)
# positive_items_per_userx is each basket of the user in each dataset trainx

In [8]:
train = pd.concat([train1, train2, train3, train4], axis = 0)[['t_dat', 'article_id', 'user_id']]
train['pop_factor'] = train['t_dat'].apply(lambda x: 1/((datetime.datetime(2020,9,23) - x).days // 7 + 1)**2)
# in this case, we give the popular index by using the week away from the end of the whole dataset.

In [9]:
# purchase count of each article
items_total_count = train.groupby(['article_id'])['article_id'].count()
# purchase count of each user
users_total_count = train.groupby(['user_id'])['user_id'].count()

In [ ]:
train['feedback'] = 1
train = train.groupby(['user_id', 'article_id']).sum().reset_index()
train['feedback'] = train.apply(lambda row: row['feedback']/popular_items_group[row['article_id']], axis=1)

train['feedback'] = train['feedback'].apply(lambda x: 5.0 if x > 5.0 else x)
train = train.sample(frac= 1).reset_index(drop=True)
train['feedback'].describe()

In [ ]:
train.to_csv('train.csv', index = False)

In [10]:
train = pd.read_csv('train.csv') 
'''
this is fast when after using the model

'''

'\nthis is fast when after using the model\n\n'

In [62]:
# get the popular items in the last month, for using in the cold recommendation
train_ = transactions[(transactions["t_dat"] >= datetime.datetime(2020,9,1)) & (transactions['t_dat'] < datetime.datetime(2020,9,23))]
train_['pop_factor'] = train_['t_dat'].apply(lambda x: 1/(datetime.datetime(2020,9,23) - x).days)
popular_items_factor = train_.groupby(['article_id'])['pop_factor'].sum()

_, popular_items = zip(*sorted(zip(popular_items_factor, popular_items_factor.keys()))[::-1])


In [25]:
user_purchase_list = train_.groupby(['user_id'])['article_id'].apply(list)

In [27]:
fm_train = train.merge(embed, on = 'article_id', how = 'left')

In [28]:
fm_train

,user_id,article_id,pop_factor,feedback,0,1,2,3,4,5,...,90,91,92,93,94,95,96,97,98,99
0,1171987,4912,0.004444,0.462161,0.077948,-0.096497,-0.105586,-0.140482,0.158139,0.037315,...,-0.078302,0.192526,0.033189,-0.071227,-0.087542,-0.064278,-0.045917,0.122057,-0.040464,-0.072226
1,1244641,44577,0.250000,3.491993,-0.013230,-0.292907,0.155605,-0.023271,-0.011968,0.102960,...,-0.112035,0.057561,-0.077107,-0.185574,0.117108,0.103868,0.133962,-0.092365,-0.056927,-0.113168
2,515438,30464,0.006920,1.357951,0.069534,-0.173100,-0.065951,0.080157,0.076465,0.118090,...,-0.120119,0.062567,-0.064975,-0.145845,0.035075,-0.003932,0.087852,0.093500,-0.012367,-0.003430
3,612287,39851,0.005102,0.030483,-0.008250,-0.088505,-0.030072,-0.008646,-0.096147,-0.102708,...,-0.093137,-0.009940,0.006413,-0.107907,-0.115632,-0.214477,-0.056602,0.019968,0.040323,0.131809
4,1229373,19563,0.005917,0.118781,-0.088854,-0.027735,0.049713,-0.199672,0.026638,0.037173,...,0.031147,0.223270,0.121350,-0.097440,-0.053039,-0.068055,-0.121871,-0.017136,-0.004563,0.006760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4963867,198142,6115,0.015625,0.002992,0.037444,0.025391,0.045489,-0.028387,-0.092784,-0.096198,...,-0.027728,-0.180711,0.039535,-0.074382,0.020917,-0.100065,-0.085434,0.075915,0.008848,0.125217
4963868,1265130,4402,0.111111,0.002547,0.036688,0.043992,-0.172384,-0.033499,-0.048964,0.020837,...,-0.141494,-0.141102,-0.014078,-0.094602,-0.136878,-0.092220,0.048717,0.000024,0.058695,0.099959
4963869,612547,28919,0.003906,0.026352,0.028164,0.049544,-0.004918,-0.026713,-0.215829,-0.045707,...,-0.028205,0.255497,-0.027412,-0.033971,-0.032903,-0.410007,0.120757,0.070377,0.143795,-0.095313
4963870,991889,36286,0.062500,0.108753,0.049940,-0.105382,-0.056156,0.032200,0.029715,0.018723,...,-0.034555,0.230959,0.122061,-0.039744,-0.026802,0.079542,0.051358,0.044519,-0.033647,0.077619


In [29]:
pickle.dump(fm_train, open(f'fm_train.pkl','wb'))

In [ ]:
fm_train = pickle.load(open("fm_train.pkl", "rb"))
'''
this is fast when after using the model

'''

In [ ]:
fm_x = fm_train.drop(['feedback', 'pop_factor'], axis = 1)
fm_y = fm_train['feedback']

In [83]:
fm_x.article_id = fm_x.article_id.astype('str')
fm_x.user_id = fm_x.user_id.astype('str')

In [73]:
from reco.recommender import FM
fm_model = FM(k = 10, learning_rate = 0.005, regularizer = .005, iterations = 1)

In [84]:
fm_model.fit(X = fm_x, y = fm_y)

epoch 0 time 804.7702125999999 mse 1.8785846014008838


In [98]:
popular_items = list(popular_items)
outputs = []
cnt = 0
user_predict = []
user = 1371960
most_common_items_of_user = {article_id: time for article_id, time in Counter(positive_items_per_user1[user]).most_common()}
# most_common_items_of_user is used to get the most frequently bought article by user
predict_df = embed.loc[embed.article_id.isin(list(most_common_items_of_user.keys())), :]
predict_df['user_id'] = str(user)
predict_df.article_id = predict_df.article_id.astype('str')
predict_df = predict_df[['user_id', 'article_id'] + list(range(100))]
fm_model.predict(predict_df)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [95]:
predict_df

,user_id,article_id,0,1,2,3,4,5,6,7,...,90,91,92,93,94,95,96,97,98,99
1,1371960,16128,0.055172,-0.033577,0.009442,-0.084133,-0.068044,-0.036341,-0.103877,0.088267,...,-0.007283,0.137015,0.032505,-0.141099,0.036039,-0.196943,0.002166,0.098618,-0.005751,0.048917
1,1371960,28040,0.285456,-0.012846,-0.025513,-0.107498,0.061365,-0.122001,-0.007056,-0.329416,...,-0.175180,0.009281,0.084295,0.208447,-0.086410,-0.165863,0.279223,0.288356,0.110291,-0.069078
4,1371960,5599,0.006237,-0.131394,-0.019968,0.039123,0.081398,-0.125605,-0.125922,0.041584,...,0.106852,0.215627,0.048003,-0.004737,0.010335,0.040623,0.028842,0.018428,0.005410,0.127457
4,1371960,26459,0.181629,-0.077577,-0.196139,0.165459,0.161856,-0.114184,-0.132380,-0.273783,...,0.056080,-0.165378,-0.003228,-0.269450,-0.214114,0.091695,0.172464,-0.082990,0.238170,0.358174
7,1371960,31011,0.109880,-0.148156,-0.022366,-0.060636,0.065297,-0.113259,-0.145037,0.071251,...,-0.004872,0.220249,0.054002,0.029841,0.070581,-0.088684,0.048450,0.003894,-0.000620,0.078128
7,1371960,31842,0.146689,-0.086021,-0.105034,0.077244,-0.121281,-0.150452,-0.004122,0.117337,...,-0.006609,-0.330679,0.006371,0.099013,0.044259,-0.078815,-0.054333,0.100142,0.052132,-0.090567


In [ ]:
'''
pickle.dump(customer_id_list, open(f'customer_id_list.pkl','wb'))
pickle.dump(pred_id_list, open(f'pred_id_list.pkl','wb'))
customer_id_list = pickle.load(open("customer_id_list.pkl", "rb"))
pred_id_list = pickle.load(open("pred_id_list.pkl", "rb"))
'''
'''
this is fast when after using the model

'''
'''
sub_pred_df_1 = pd.DataFrame({'customer_id': customer_id_list, 'prediction': pred_id_list})
'''

## preprocess customer_df

In [ ]:
def preprocessing_customer_df(customer_df):
    # change customer_id to user_id in customer information dataframe
    customer_df['user_id'] = customer_df['customer_id'].map(customer_to_index_map)
    customer_df.drop(['customer_id'], axis = 1, inplace = True)

    customer_df['user_id'] = pd.Categorical(customer_df['user_id'])
    
    # convert object variables to category type
    categorical_columns = customer_df.select_dtypes(include = 'object').columns
    for categorical_column in categorical_columns:
        customer_df[categorical_column] = pd.Categorical(customer_df[categorical_column])
    # print(customer_df.isnull().sum() / customer_df.shape[0])
    
    # drop FN, Active
    customer_df = customer_df.drop(['FN', 'Active'], axis = 1)
    
    # fill nan in club_member_status
    # print(customer_df.groupby(['club_member_status']).count())
    np.random.seed(0)
    p = np.array([1260566/1351944, 467/1351944, 90911/1351944])
    index = np.random.choice(['ACTIVE', 'LEFTCLUB', 'PRE-CREATE'], p = p.ravel(), size = 6062)
    customer_club_null = customer_df[customer_df['club_member_status'].isnull()]
    customer_club_not_null = customer_df[~customer_df['club_member_status'].isnull()]
    customer_club_null['club_member_status'] = index
    customer_df = pd.concat([customer_club_not_null, customer_club_null], axis = 0)
    
    # fill nan in fashion_news_frequency
    # print(customer_df.fashion_news_frequency.value_counts())
    customer_fashion_null = customer_df[customer_df['fashion_news_frequency'].isnull()]
    p = np.array([877713/1355971, 477416/1355971, 842/1355971])
    index = np.random.choice(['NONE', 'Regularly', 'PRE-Monthly'], p = p.ravel(), size = 16009)
    customer_fashion_not_null = customer_df[~ customer_df['fashion_news_frequency'].isnull()]
    customer_fashion_null['fashion_news_frequency'] = index
    customer_df = pd.concat([customer_fashion_not_null, customer_fashion_null], axis = 0)
    
    # NONE and None can be guessed as the same
    customer_fashion_None = customer_df[customer_df['fashion_news_frequency'] == 'None']
    customer_fashion_not_None = customer_df[customer_df['fashion_news_frequency'] != 'None']
    customer_fashion_None['fashion_news_frequency'] = 'NONE'
    customer_df = pd.concat([customer_fashion_None, customer_fashion_not_None], axis = 0)
    
    # LEFTCLUB and LEFT CLUB can be guessed as the same
    customer_club_LEFTCLUB = customer_df[customer_df['club_member_status'] == 'LEFTCLUB']
    customer_club_not_LEFTCLUB = customer_df[customer_df['club_member_status'] != 'LEFTCLUB']
    customer_club_LEFTCLUB['club_member_status'] = 'LEFT CLUB'
    customer_df = pd.concat([customer_club_LEFTCLUB, customer_club_not_LEFTCLUB], axis = 0)
    
    # fill nan in age columns by using the mean of that grouped by cllub_member_status
    customer_age_group = customer_df.groupby('club_member_status')['age'].mean().to_dict()
    # print(customer_age_group)
    customer_age_null = customer_df[customer_df['age'].isnull()]
    customer_age_not_null = customer_df[~ customer_df['age'].isnull()]
    customer_age_null_ACTIVE = customer_age_null[customer_age_null['club_member_status'] == 'ACTIVE'] 
    customer_age_null_ACTIVE['age'] = customer_age_group['ACTIVE']
    customer_age_null_LEFTCLUB = customer_age_null[customer_age_null['club_member_status'] == 'LEFT CLUB'] 
    customer_age_null_LEFTCLUB['age'] = customer_age_group['LEFT CLUB']
    customer_age_null_PRECREATE = customer_age_null[customer_age_null['club_member_status'] == 'PRE-CREATE'] 
    customer_age_null_PRECREATE['age'] = customer_age_group['PRE-CREATE']
    customer_df = pd.concat([customer_age_null_ACTIVE, customer_age_null_LEFTCLUB], axis = 0)
    customer_df = pd.concat([customer_df, customer_age_null_PRECREATE], axis = 0)
    customer_df = pd.concat([customer_df, customer_age_not_null], axis = 0)
    
   # drop postal_code since it is too much and useless
    customer_df = customer_df.drop('postal_code', axis = 1)
    # get dummies of club_member_status and fashion_news_frequencydummy
    for target_category_col in ['club_member_status', 'fashion_news_frequency']:
        customer_df[target_category_col] = label_encoder.fit_transform(customer_df[target_category_col])
        customer_df[target_category_col] = pd.Categorical(customer_df[target_category_col])
    customer_df_feature = customer_df[['club_member_status', 'fashion_news_frequency', 'age']]
    customer_df_id = customer_df['user_id']
    customer_df_feature = pd.get_dummies(customer_df_feature, prefix = ['club_member_status', 'fashion_news_frequency'])
    customer_df = pd.concat([customer_df_feature, customer_df_id], axis = 1)
    return customer_df
customer_df = preprocessing_customer_df(customer_df)

In [ ]:
pickle.dump(customer_df, open(f'customer_df.pkl','wb'))

In [139]:
customer_df = pickle.load(open("customer_df.pkl", "rb"))
'''
this is fast when after using the model

'''

'\nthis is fast when after using the model\n\n'

## combine the dataset and FM part

In [ ]:
'''
customer_df = customer_df.sort_values(by = 'user_id')
embed = embed.sort_values(by = 'article_id').set_index('article_id').reset_index()
dataset = Dataset()
dataset.fit(users = customer_df['user_id'], items = embed['article_id'], 
            item_features = new_col)
num_users, num_topics = dataset.interactions_shape()
print(f'Number of users: {num_users}, Number of topics: {num_topics}.')
(interactions, weights) = dataset.build_interactions(transactions[['user_id', 'article_id']].values)
user_features = csr_matrix(customer_df.drop('user_id', axis = 1))
item_features = csr_matrix(embed.drop('article_id', axis = 1))
data_1 = np.ones(num_users)
row_1 = np.ones(num_users)
col_1 = np.ones(num_users)
customer_sp_csr_1 = sparse.coo_matrix((data, (row, col)), shape = (num_users, num_users)).tocsr()
data_2 = np.ones(num_topics)
row_2 = np.ones(num_topics)
col_2 = np.ones(num_topics)
customer_sp_csr_2 = sparse.coo_matrix((data, (row, col)), shape = (num_topics, num_topics)).tocsr()
item_features_ = sparse.hstack([customer_sp_csr_2, item_features])
modelFM = LightFM(loss='warp', learning_rate = 0.001, random_state = np.random.RandomState(123))
modelFM.fit(interactions = interactions, user_features = user_features_,
            item_features = item_features_,  epochs = 1)
pickle.dump(modelFM, open(DATA_PATH + 'lightFM1.pickle', "wb"))
'''

In [ ]:
'''
sub_customer_id = sub_df.customer_id.map(customer_to_index_map)
sub_customer = sub_df.customer_id
def get_predictions(customer_id):
    pred = modelFM.predict(customer_id, np.arange(embed.shape[0]), user_features = user_features_,
            item_features = item_features_)
    predict_list = np.argsort(- pred)[:12]
    result_list = [lbe_dict['article_id'].inverse_transform([item_id])[0] for item_id in predict_list]
    preds = ' '.join(result_list)
    return preds
customer_id_list_2 = []
pred_id_list_2 = []
for cus_id_ in tqdm.tqdm(set(sub_customer_id) - set(df_corpus.index)):
    customer_id_list_2.append(get_predictions(cus_id_))
    pred_id_list_2.append(sub_customer[cus_id_])
customer_id_list_2_ = customer_id_list_2.copy()
pred_id_list_2_ = pred_id_list_2.copy()
pickle.dump(customer_id_list_2, open(f'customer_id_list_2.pkl','wb'))
pickle.dump(pred_id_list_2, open(f'pred_id_list_2.pkl','wb'))
'''

In [ ]:
'''
customer_id_list_2 = pickle.load(open("customer_id_list_2.pkl", "rb"))
pred_id_list_2 = pickle.load(open("pred_id_list_2.pkl", "rb"))
'''
'''
this is fast when after using the model

'''

In [ ]:
'''
sub_pred_df_2 = pd.DataFrame({'prediction': customer_id_list_2, 'customer_id': pred_id_list_2})
# prediction and customer_id seem that are converted...., so in sub_pred_df we change that in the true columns
sub_pred_df = pd.concat([sub_pred_df_1, sub_pred_df_2], axis = 0)
sub_pred_df.to_csv('submission.csv', index = False)
'''